# Setup environment

###Load TokaMaker and helpful python packages
In this code segment, we import several helpful python packages and load in the TokaMaker code from GitHub. Default plotting values are also set here to make things more legible on most platforms. We also define a resize_polygon function which is helpful for defining geometries later on. **You do not need to change this code.**

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

plt.rcParams['figure.figsize']=(6,6)
plt.rcParams['font.weight']='bold'
plt.rcParams['axes.labelweight']='bold'
plt.rcParams['lines.linewidth']=2
plt.rcParams['lines.markeredgewidth']=2

In [2]:
# Get the home directory dynamically
home_dir = os.path.expanduser("~")

# Construct the path dynamically
oft_root_path = os.path.join(home_dir, "OpenFUSIONToolkit/install_release")

# Set the environment variable
os.environ["OFT_ROOTPATH"] = oft_root_path

# Append to sys.path
tokamaker_python_path = os.getenv("OFT_ROOTPATH")
if tokamaker_python_path is not None:
    sys.path.append(os.path.join(tokamaker_python_path, "python"))


In [3]:
from OpenFUSIONToolkit import OFT_env
from OpenFUSIONToolkit.TokaMaker import TokaMaker
from OpenFUSIONToolkit.TokaMaker.meshing import gs_Domain, save_gs_mesh, load_gs_mesh
from OpenFUSIONToolkit.TokaMaker.util import create_isoflux, create_power_flux_fun

In [4]:
def resize_polygon(points, dx):
    new_points = np.empty(np.shape(points))
    for i in range(np.shape(points)[0]):
        if i==0:
            last = points[-1,:]
            next = points[i+1,:]
        elif i == np.shape(points)[0]-1:
            last = points[i-1,:]
            next = points[0,:]
        else:
            next = points[i+1,:]
            last = points[i-1,:]
        par = points[i,:]-last
        par/= np.linalg.norm(par)
        perp = np.array([par[1], -par[0]])
        temp = points[i,:] + perp*dx
        par_2 = next-points[i,:]
        par_2/= np.linalg.norm(par_2)
        perp_2 = [par_2[1], -par_2[0]]
        new_points[i, :] = temp + dx/np.dot(perp_2,par)*par  + par*dx/np.dot(par_2,perp)*np.dot(par_2,par)
    return new_points

In [5]:
import json
import glob
import os
from datetime import datetime

# Create examples directory if it doesn't exist
examples_dir = "examples"
if not os.path.exists(examples_dir):
    os.makedirs(examples_dir)
    print(f"Created {examples_dir} directory")

# For testing, just use a simple folder name
folder_name = "testing_1"
print(f"Using folder name: {folder_name}")

# Create the simulation folder (no timestamp for simplicity)
simulation_folder = os.path.join(examples_dir, folder_name)

# Create the simulation folder
os.makedirs(simulation_folder, exist_ok=True)
print(f"Created simulation folder: {simulation_folder}")

# Function to save figures
def save_figure(fig, filename, folder=simulation_folder):
    """Save figure to the simulation folder"""
    filepath = os.path.join(folder, filename)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    print(f"Saved: {filepath}")

Using folder name: testing_1
Created simulation folder: examples/testing_1


# Setup device geometry
First, we need to tell TokaMaker what our tokamak should look like!

### Set mesh resolution
TokaMaker solves the Grad-Shafranov equation on a computational mesh. The finer the mesh is, the more accurate the solution will be. However, a finer mesh also decreases the speed of the solver. Here, we set a default resolution for the mesh in each domain. **You do not need to change these values.**

In [6]:
#### DO NOT CHANGE ####
plasma_dx = 0.15
coil_dx = 0.15
vv_dx = 0.15
vac_dx = 0.25
#### DO NOT CHANGE ####

### Specify the geometry of the vacuum vessel

We need to decide on the shape of our vacuum vessel. We want to design a vacuum vessel that will fit our desired plasma shape. To visualize what our plasma looks like, we can use several quantities on your list of targets, including major radius, minor radius, elongation, and triangularity. Modify these quantities to correspond to your targets



In [7]:


# Find all JSON files in the directory
json_files = glob.glob('examples/testing_1/*.json')

if json_files:
    # Sort to get the most recent file (if you want the latest)
    json_files.sort(reverse=True)
    latest_file = json_files[0]
    
    # Now open the specific file
    with open(latest_file, 'r') as f:
        design_data = json.load(f)
        print(f"Loaded data from: {latest_file}")
        major_radius = design_data['plasma_parameters']['major_radius']
        minor_radius = design_data['plasma_parameters']['minor_radius']
        elongation = design_data['plasma_parameters']['elongation']
        triangularity = design_data['plasma_parameters']['triangularity']
        print(f"major_radius: {major_radius}, minor_radius: {minor_radius}, elongation: {elongation}, triangularity: {triangularity}")
else:
    print("No JSON files found")

boundary_pts = create_isoflux(30,major_radius,0.0,minor_radius,elongation,triangularity)

Loaded data from: examples/testing_1/design_20250702_113339.json
major_radius: 4.75, minor_radius: 1.05, elongation: 1.65, triangularity: -0.4


Now, we have to decide the shape of our vacuum vessel cross-section. We do this by defining an array of (R,Z) coordinates for the vacuum vessel boundary. Every number in the array must be a float (a number including a decimal point). So, 3.5 and 3.0 are acceptable entries, but 3 is not.

In [8]:

vv_boundary = np.array(design_data['vacuum_vessel']['boundary_coordinates'])
print(vv_boundary)

''' 
NEGATIVE TRIANGULARITY VV 
vv_boundary = np.array([[3.25, -0.75],[3.75, -1.25], [4.5, -1.85],[6.0, -1.85], [6.0, 1.85], [4.5, 1.85], [3.75, 1.25], [3.25, 1.0]]) ##Add/change the vacuum vessel coordinates here!!!

BOX VV
vv_boundary = np.array([[3.2, -1.8], [6.0, -1.8], [6.0, 1.8], [3.2,1.8]]) 
'''

vv_outer = resize_polygon(vv_boundary, 0.04)

[[ 3.25 -0.75]
 [ 3.5  -1.5 ]
 [ 4.5  -1.85]
 [ 6.   -1.85]
 [ 6.    1.85]
 [ 4.5   1.85]
 [ 3.5   1.5 ]
 [ 3.25  1.  ]]


Now, you can plot the shape of the plasma and the vacuum vessel boundary to see what they look like.  If your plasma cross-section does not fit inside the vacuum vessel, go back and modify your vacuum vessel points!

In [9]:
fig, ax = plt.subplots()
ax.fill(vv_outer[:,0], vv_outer[:,1], color = 'k')
ax.fill(vv_boundary[:,0], vv_boundary[:,1], color = 'w')
ax.scatter(boundary_pts[:,0], boundary_pts[:,1], color = 'b')
ax.set_aspect(aspect = 1)
ax.set_xlabel('R (m)')
ax.set_ylabel('Z (m)')

Text(0, 0.5, 'Z (m)')

### Place poloidal field coils
Now, we need to specify the locations of our poloidal field coils using (R,Z) coordinates. You can use a total of 8 poloidal field coils. You will need to change these coordinates to achieve the desired plasma shape.

In [10]:
'''
default coils 

PF_1_R = 4.0
PF_1_Z = 2.0

PF_2_R = 4.0
PF_2_Z = -2.0

PF_3_R = 5.25
PF_3_Z = 2.25

PF_4_R = 5.25
PF_4_Z = -2.25

PF_5_R = 6.5
PF_5_Z = .5

PF_6_R = 6.5
PF_6_Z = -.5

PF_7_R = 3
PF_7_Z = 0.25

PF_8_R = 3
PF_8_Z = -0.25
'''

# coil_locs = np.array([[PF_1_R, PF_1_Z], [PF_2_R, PF_2_Z], [PF_3_R, PF_3_Z], [PF_4_R, PF_4_Z], [PF_5_R, PF_5_Z], [PF_6_R, PF_6_Z], [PF_7_R, PF_7_Z], [PF_8_R, PF_8_Z]])

coil_locs = np.array(design_data["coil_coordinates"])

print(coil_locs)

[[ 4.    2.5 ]
 [ 4.   -2.5 ]
 [ 5.25  2.25]
 [ 5.25 -2.25]
 [ 6.5   0.5 ]
 [ 6.5  -0.5 ]
 [ 2.8   0.25]
 [ 2.8  -0.25]]


We can plot the locations of our coils and the vacuum vessel boundary. Make sure that the coils are outside of the vacuum vessel.

In [11]:
fig, ax = plt.subplots()
ax.fill(vv_outer[:,0], vv_outer[:,1], color = 'k')
ax.fill(vv_boundary[:,0], vv_boundary[:,1], color = 'w')
ax.scatter(boundary_pts[:,0], boundary_pts[:,1], color = 'b')
ax.scatter(coil_locs[:,0], coil_locs[:,1], color = 'r')
ax.set_aspect(aspect = 1)
ax.set_xlabel('R (m)')
ax.set_ylabel('Z (m)')

plt.show()
save_figure(fig, "01_vacuum_vessel_design.png")

Saved: examples/testing_1/01_vacuum_vessel_design.png


/var/folders/z5/d2ytmy3d2h18qcqwb9v71w2m0000gq/T/ipykernel_36889/2326095319.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Define the regions of our tokamak
We have to tell the solver what domains we plan on implementing in our tokamak. For this device, we implement four types of domains:
1.   An air domain, which surrounds the tokamak
2.   A vaccum vessel domain, for the vacuum vessel containing the plasma
3.   A plasma domain, where the plasma will be located
4.   Eight poloidal field coil domains, which contain the coils we will use to shape the plasma

**You do not need to change this code.**



In [12]:
#### DO NOT CHANGE ####
# Create a mesh object
mesh = gs_Domain()
# Define region information for mesh
mesh.define_region('air',vac_dx,'boundary')        # Air domain
mesh.define_region('vv',vv_dx,'conductor',eta= 6.9E-7)      # Vaccum vessel domain
mesh.define_region('plasma',plasma_dx,'plasma')    # Plasma domain
# Define each of the PF coils
for i in range(1,9):
    mesh.define_region('PF_' + str(i),coil_dx,'coil') # Coil domains
#### DO NOT CHANGE ####

# only works for 8 coils

## Add region boundaries to our gs_Domain object
Now that we have decided on our geometry, we need to pass that information on to the mesh object. **You do not need to modify this code.**

In [13]:
#### DO NOT CHANGE ####
# Define geometry
mesh.add_annulus(vv_boundary,'plasma',vv_outer,'vv',parent_name='air') # Define the shape of the VV

# Define the shape of the PF coils
for i in range(1,9):
    mesh.add_rectangle(coil_locs[i-1,0],coil_locs[i-1,1],0.3,0.3,'PF_' + str(i),parent_name='air')
#### DO NOT CHANGE ####

In [14]:
fig, ax = plt.subplots(1,1,figsize=(4,6),constrained_layout=True)
mesh.plot_topology(fig,ax)

### Generate mesh
Now we generate the actual mesh that TokaMaker will use to solve the Grad Shafranov equation. We also plot the mesh to make sure that each region is defined properly. **You do not need to change this code.**

In [15]:
#### DO NOT CHANGE ####
mesh_pts, mesh_lc, mesh_reg = mesh.build_mesh()
coil_dict = mesh.get_coils()
cond_dict = mesh.get_conductors()
#### DO NOT CHANGE ####

print(coil_dict)
print(cond_dict)

Assembling regions:
  # of unique points    = 321
  # of unique segments  = 46


Generating mesh:
  # of points  = 1953
  # of cells   = 3792
  # of regions = 11
{'PF_1': {'reg_id': 4, 'coil_id': 0, 'nturns': 1, 'coil_set': 'PF_1', 'allow_xpoints': False}, 'PF_2': {'reg_id': 5, 'coil_id': 1, 'nturns': 1, 'coil_set': 'PF_2', 'allow_xpoints': False}, 'PF_3': {'reg_id': 6, 'coil_id': 2, 'nturns': 1, 'coil_set': 'PF_3', 'allow_xpoints': False}, 'PF_4': {'reg_id': 7, 'coil_id': 3, 'nturns': 1, 'coil_set': 'PF_4', 'allow_xpoints': False}, 'PF_5': {'reg_id': 8, 'coil_id': 4, 'nturns': 1, 'coil_set': 'PF_5', 'allow_xpoints': False}, 'PF_6': {'reg_id': 9, 'coil_id': 5, 'nturns': 1, 'coil_set': 'PF_6', 'allow_xpoints': False}, 'PF_7': {'reg_id': 10, 'coil_id': 6, 'nturns': 1, 'coil_set': 'PF_7', 'allow_xpoints': False}, 'PF_8': {'reg_id': 11, 'coil_id': 7, 'nturns': 1, 'coil_set': 'PF_8', 'allow_xpoints': False}}
{'AIR': {'reg_id': 2, 'vac_id': 0, 'allow_xpoints': False}, 'VV': {'reg_id': 3, 'cond_id': 0, 'eta': 6.9e-07, 'noncontinuous': False, 'allow_xpoints': False}}


In [16]:
fig, ax = plt.subplots(1,1,figsize=(5,5),constrained_layout=True)
mesh.plot_mesh(fig,ax)

# Find a plasma equilibrium

### Setup TokaMaker
Now that we have set up our device, we are ready to solve for an equilibrium. First, we need to make a new object of the TokaMaker class and load in our mesh information.

**Important note:** Without resetting your tokamaker object, these code segments can only be run once per Google colab session.  If you have modified your device and would like to run these lines again, please uncomment the "tokamaker.reset()" line, which will allow you to run tokamaker.setup() again. If you accidentally run these cells again without running reset(), you may get a message that your kernel has crashed. You can go to 'Runtime' -> 'Restart Session' in the Google Colab menu to start a new session (all of your code will be saved).

In [17]:
# tokamaker.reset()

In [18]:
#### DO NOT CHANGE ####
myOFT = OFT_env(nthreads=2)
tokamaker = TokaMaker(myOFT)
tokamaker.setup_mesh(mesh_pts, mesh_lc, mesh_reg)
tokamaker.setup_regions(cond_dict=cond_dict,coil_dict=coil_dict)
#### DO NOT CHANGE ####

#----------------------------------------------
Open FUSION Toolkit Initialized
Development branch:   main
Revision id:          f8ced65
Parallelization Info:
  Not compiled with MPI
  # of OpenMP threads =    2
Fortran input file    = /var/folders/z5/d2ytmy3d2h18qcqwb9v71w2m0000gq/T/oft_36889/oftpyin
XML input file        = none
Integer Precisions    =    4   8
Float Precisions      =    4   8  16
Complex Precisions    =    4   8
LA backend            = native
#----------------------------------------------


**** Generating surface grid level  1
  Generating boundary domain linkage
  Mesh statistics:
    Area         =  5.075E+01
    # of points  =    1953
    # of edges   =    5744
    # of cells   =    3792
    # of boundary points =     112
    # of boundary edges  =     112
    # of boundary cells  =     112
  Resolution statistics:
    hmin =  4.000E-02
    hrms =  1.869E-01
    hmax =  3.892E-01
  Surface grounded at vertex     309



### Set magnetic field and major radius

Here, we specify the magnetic field and major radius of the plasma. You should change these values to match your target parameters.

In [19]:

# NEGATIVE TRIANGULARITY
B0 = design_data["advanced_settings"]["B0"] #toroidal magnetic field

# POSITIVE TRIANGULARITY
# B0 = 1 #toroidal magnetic field

R0 = major_radius #major radius
tokamaker.setup(order=2,F0=B0*R0)




**** Creating Lagrange FE space
  Order  =    2
  Minlev =   -1

 Computing flux BC matrix 


 Inverting real matrix
   Time =    3.7740000000000000E-003


### Define global quantities and targets

We need to tell the solver a bit more information about our device. First, we need to specify the total current in the plasma, by setting a target value for 'Ip'.

We also need to specify a target for the plasma pressure in the core of the tokamak. To do this, we set a number for the 'p_core_target'.

You will likely need to adjust these quantities to match other design targets, such as an appropriate q-profile and the correct poloidal beta ($\beta_p$).

In [20]:
# POSITIVE TRIANGULARITY

# Ip_target = 3E6
# Ip_ratio_target = 1.0


# NEGATIVE TRIANGULARITY
# Ip_target = 8E6
# Ip_ratio_target = 0.333

Ip_target = design_data["advanced_settings"]["Ip_target"]
Ip_ratio_target = design_data["advanced_settings"]["Ip_ratio_target"]

tokamaker.set_targets(Ip=Ip_target,Ip_ratio = Ip_ratio_target)

### Define plasma profiles

The magnitudes of the F*F' and P' profiles are determined by the global quantities set above.  Now, we need to specify the actual shape of the profiles. For simplicity, we will use simple polynomial profiles of the form
 $((1-\hat{\psi})^{\alpha})^{\gamma}$ using the built-in TokaMaker function 'create_power_flux_fun'.

 Feel free to adjust the alpha ($\alpha$) and gamma ($\gamma$) parameters for both P' and F*F' to reach your desired internal inductance ($l_i$) target.


In [21]:


# NEGATIVE TRIANGULARITY
# ffp_alpha = 2.15
# ffp_gamma = 1.7

# pp_alpha = 2.15
# pp_gamma = 1.7

# POSITIVE TRIANGULARITY

# ffp_alpha = 4.0
# ffp_gamma = 1.0

# pp_alpha = 1.5
# pp_gamma = 2.0

ffp_alpha = design_data["advanced_settings"]["ffp_alpha"]
ffp_gamma = design_data["advanced_settings"]["ffp_gamma"]
pp_alpha = design_data["advanced_settings"]["pp_alpha"]
pp_gamma = design_data["advanced_settings"]["pp_gamma"]

ffp_prof = create_power_flux_fun(40,pp_alpha,pp_gamma)
pp_prof = create_power_flux_fun(40,ffp_alpha,ffp_gamma)

tokamaker.set_profiles(ffp_prof=ffp_prof,pp_prof=pp_prof)

In [22]:
'''
major_radius = 4.55
minor_radius = 1.2
elongation = 1.4
triangularity =  -0.5
'''

# reads the json file in testing_1

parameter_json = glob.glob('tokamak_psp_2025/examples/testing_1/*.json')

try:
    with open(parameter_json, 'r') as f:
        design_data = json.load(f)
        print("✅ Loaded design parameters from json")
except Exception as e:
        print(f"❌ Error loading json file: {e}")


major_radius = design_data['plasma_parameters']['major_radius']
minor_radius = design_data['plasma_parameters']['minor_radius']
elongation = design_data['plasma_parameters']['elongation']
triangularity = design_data['plasma_parameters']['triangularity']

boundary_pts = create_isoflux(30,major_radius,0.0,minor_radius,elongation,triangularity)

❌ Error loading json file: unhashable type: 'list'


Now, we can visualize our profiles. "Peaked" profiles, which are more concentrated around $\Psi$ = 0, correspond to higher internal inductance ($l_i$).

In [23]:
fig, ax = plt.subplots(2,1,sharex=True)
# Plot F*F'
ax[0].plot(ffp_prof['x'],ffp_prof['y'])
ax[0].set_ylabel("FF'")
# Plot P'
ax[1].plot(pp_prof['x'],pp_prof['y'])
ax[1].set_ylabel("P'")
_ = ax[-1].set_xlabel(r"$\hat{\psi}$")

plt.show()
save_figure(fig, "02_plasma_profiles.png")


Saved: examples/testing_1/02_plasma_profiles.png


/var/folders/z5/d2ytmy3d2h18qcqwb9v71w2m0000gq/T/ipykernel_36889/373337011.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Set shape targets

Now we need to tell TokaMaker what the shape of our plasma cross-section will look like. Given that we've already made an array of points defining our plasma, we just need to pass this array to the set_isoflux() method. **You do not need to change this code**.

In [24]:
tokamaker.set_isoflux(boundary_pts, weights = 5*np.ones(len(boundary_pts)))

### Set up coil regularization

This code segment makes sure that the solver tries to minimize the currents in each coil. **You should not change this code**

In [25]:
#### DO NOT CHANGE ####
coil_regmat = np.zeros((tokamaker.ncoils+1,tokamaker.ncoils+1), dtype=np.float64)
coil_regmat = np.eye(tokamaker.ncoils+1, dtype = np.float64)
targets = np.zeros(tokamaker.ncoils+1)
weights = 0.1*np.ones(tokamaker.ncoils+1)
tokamaker.set_coil_reg(coil_regmat, targets, weights)
#### DO NOT CHANGE ####

Now we are ready to solve. **You shouldn't have to edit any of this code.**

In [26]:
#### DO NOT CHANGE ####
tokamaker.init_psi(major_radius,0.0,minor_radius,elongation,triangularity)
err_flag = tokamaker.solve()
#### DO NOT CHANGE ####

Starting non-linear GS solver
     1  6.3563E+00  4.0685E-01  1.9299E-01  4.8967E+00 -4.3116E-04 -0.0000E+00
     2  7.4653E+00  4.7701E-01  9.8942E-02  4.9003E+00 -3.2201E-04 -0.0000E+00
     3  8.0571E+00  5.1494E-01  5.1895E-02  4.9010E+00 -2.4206E-04 -0.0000E+00
     4  8.4035E+00  5.3738E-01  2.9338E-02  4.9011E+00 -2.0175E-04 -0.0000E+00
     5  8.6134E+00  5.5108E-01  1.7483E-02  4.9009E+00 -1.7316E-04 -0.0000E+00
     6  8.7438E+00  5.5964E-01  1.0710E-02  4.9006E+00 -1.5655E-04 -0.0000E+00
     7  8.8247E+00  5.6497E-01  6.6378E-03  4.9005E+00 -1.4745E-04 -0.0000E+00
     8  8.8752E+00  5.6831E-01  4.1507E-03  4.9003E+00 -1.2776E-04 -0.0000E+00
     9  8.9069E+00  5.7041E-01  2.6000E-03  4.9002E+00 -1.2365E-04 -0.0000E+00
    10  8.9267E+00  5.7172E-01  1.6321E-03  4.9002E+00 -1.2087E-04 -0.0000E+00
    11  8.9391E+00  5.7255E-01  1.0261E-03  4.9001E+00 -1.1890E-04 -0.0000E+00
    12  8.9470E+00  5.7307E-01  6.4527E-04  4.9001E+00 -1.1753E-04 -0.0000E+00
    13  8.9519E+00  5.

### Check for convergence
This code prints out err_flag. If it is anything other than 0, the solver has failed.

In [27]:
print(err_flag)
tokamaker.print_info()
eq = tokamaker.get_stats()
print(eq)

None
Equilibrium Statistics:
  Topology                =   Limited
  Toroidal Current [A]    =    8.0000E+06
  Current Centroid [m]    =    4.876 -0.000
  Magnetic Axis [m]       =    4.900 -0.000
  Elongation              =    1.715 (U:  1.716, L:  1.715)
  Triangularity           =   -0.433 (U: -0.432, L: -0.433)
  Plasma Volume [m^3]     =   168.668
  q_0, q_95               =    1.280  2.948
  Peak Pressure [Pa]      =    1.4657E+06
  Stored Energy [J]       =    9.8792E+07
  <Beta_pol> [%]          =   75.5687
  <Beta_tor> [%]          =    0.8155
  <Beta_n>   [%]          =    1.1487
  Diamagnetic flux [Wb]   =    8.3240E-02
  Toroidal flux [Wb]      =    6.0425E+01
  l_i                     =    1.0121
{'Ip': 7999993.000271447, 'Ip_centroid': array([ 4.87646220e+00, -1.98709599e-06]), 'kappa': np.float64(1.7151993334435267), 'kappaU': np.float64(1.7156990745564007), 'kappaL': np.float64(1.7146995923306525), 'delta': np.float64(-0.43257794391620674), 'deltaU': np.float64(-0.43181

# Analyze equilibrium

Now we are ready to analyze our equilibrium and check how it performs against our desired metrics.

### Plot equilibrium solution

The red crosses correspond to our target plasma boundary. Check if this lines up with the actual plasma boundary. If they are not similar, you may need to adjust your coil positions to create your desired shape.

**You do not need to change this code.**


In [28]:
#### DO NOT CHANGE ####
fig, ax = plt.subplots()
tokamaker.plot_machine(fig,ax)
tokamaker.plot_psi(fig,ax)
tokamaker.plot_constraints(fig,ax)

ax.set_xlabel('R (m)')
ax.set_ylabel('Z (m)')
#### DO NOT CHANGE ####

plt.show()
save_figure(fig, "03_initial_equilibrium.png")

Saved: examples/testing_1/03_initial_equilibrium.png


/var/folders/z5/d2ytmy3d2h18qcqwb9v71w2m0000gq/T/ipykernel_36889/2784647179.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Print equilibrium statistics

TokaMaker has a built-in function to compute a bunch of relevant quantities for an equilibrium. There are a few things you should take note of:

1.   Where is the current centroid? Is it close to your desired major radius? If not, you may need to adjust your coil positions to better match the target shape. (Cell 9)
2.   What is the elongation? Is it close to your target elongation? If not, you may need to adjust your coil positions to better match the target shape. (Cell 9)
3.   What is the triangularity. Is it close to the target triangularity? If not, you may need to adjust your coil positions to better match the target shape. (Cell 9)
4.   What is your q_95? Is it less than 2.5? If so, you may need to decrease your target plasma current (Ip) to increase this number.  Is it way above 4? If so, you may want to increase your plasma current to decrease this number. (Cell 19)
5. What is your beta_pol? Is it close to your target beta_pol? If not, you might need to adjust your p_core_target. (Cell 19)
6. What is your internal inductance(l_i) ? Is it close to your target l_i? If not, you may need to alter the alpha and gamma factors in your FF' and P' profile shapes to alter this number. (Cell 20)



In [29]:
tokamaker.get_stats()

{'Ip': 7999993.000271447,
 'Ip_centroid': array([ 4.87646220e+00, -1.98709599e-06]),
 'kappa': np.float64(1.7151993334435267),
 'kappaU': np.float64(1.7156990745564007),
 'kappaL': np.float64(1.7146995923306525),
 'delta': np.float64(-0.43257794391620674),
 'deltaU': np.float64(-0.4318112281375407),
 'deltaL': np.float64(-0.4333446596948736),
 'R_geo': np.float64(4.763048247627427),
 'a_geo': np.float64(1.027182254820414),
 'vol': 168.66811784101137,
 'q_0': np.float64(1.2801828611444062),
 'q_95': np.float64(2.9482940101169954),
 'P_ax': np.float64(1465720.661883168),
 'W_MHD': 98791616.93675931,
 'beta_pol': np.float64(75.568725100959),
 'dflux': 0.08323995432370192,
 'tflux': 60.42496513211805,
 'l_i': np.float64(1.012057144499605),
 'beta_tor': np.float64(0.8155170918179001),
 'beta_n': np.float64(1.148662075827516)}

We can also save all of this equilibrium information in a python dictionary, so we can access it later if needed.

In [30]:
eq_info = tokamaker.get_stats()
print(eq_info.keys())
print(eq_info["beta_n"])
print(eq_info["beta_pol"])
print(eq_info["beta_tor"])

dict_keys(['Ip', 'Ip_centroid', 'kappa', 'kappaU', 'kappaL', 'delta', 'deltaU', 'deltaL', 'R_geo', 'a_geo', 'vol', 'q_0', 'q_95', 'P_ax', 'W_MHD', 'beta_pol', 'dflux', 'tflux', 'l_i', 'beta_tor', 'beta_n'])
1.148662075827516
75.568725100959
0.8155170918179001


### Plot coil currents

For our device to be efficient, we want the currents in our poloidal field coils to be low. Here, we check if our poloidal field coil currents are more than two times the plasma current by plotting our coil currents and the current limit (dashed red line).

If any coil has current greater than two times the plasma current, try to adjust your coil positions to reduce their currents. Some things to try:


*   Move your coils closer to the plasma (this may require modifying your vacuum vessel)
*   Move your coils apart from one another, so they don't have to compete



In [31]:
fig,ax = plt.subplots(figsize = (8,6))

coil_currents, current_map = tokamaker.get_coil_currents()

for i, key in enumerate(coil_dict.keys()):
    ax.scatter(key, np.abs(coil_currents[key] / 1E6), color='tab:blue')

coil_current_limit = eq_info['Ip']*2
ax.hlines(coil_current_limit/1E6, xmin = 0, xmax = len(coil_dict.keys()), color = 'r', linestyle = '--')

ax.set_xlabel('Coil name')
ax.set_ylabel('Coil current (MA)')

print(coil_currents)

plt.show()
save_figure(fig, "04_coil_currents.png")

/var/folders/z5/d2ytmy3d2h18qcqwb9v71w2m0000gq/T/ipykernel_36889/4287444784.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


{'PF_1': np.float64(-1691280.4093849463), 'PF_2': np.float64(-1690813.328029844), 'PF_3': np.float64(1140850.62707465), 'PF_4': np.float64(1139846.2942313638), 'PF_5': np.float64(-1870503.921520415), 'PF_6': np.float64(-1870355.1280032427), 'PF_7': np.float64(-331177.0563966412), 'PF_8': np.float64(-332322.9172698995)}


Saved: examples/testing_1/04_coil_currents.png


### Check vertical stability

Plasmas in tokamaks with elongation greater than 1 can be susceptible to a "vertical instability", where the plasma moves up or down within the vacuum vessel at an exponential rate. Here, we check whether the growth rate of our vertical instability is low enough that it can be prevented with a control system.

If the "feedback capability parameter" is greater than 2, modify your vacuum vessel to make it closer to the plasma, because the vaccum vessel can help provide vertical stability.

**You do not need to change this code.**

In [32]:
#### DO NOT CHANGE ####
eig_vals, eig_vecs = tokamaker.eig_td(omega = -1E4, neigs = 10)
growth_rate = -eig_vals[0,0]

eigval_wall, eigvec_wall = tokamaker.eig_wall()
wall_time = 1/eigval_wall[1,0]

feedback_capability_param = growth_rate*wall_time
print('Feedback capability parameter: ' + str(feedback_capability_param))
#### DO NOT CHANGE ####

Feedback capability parameter: 1.9673539308010533


# Calculating Vertical stability in CUTE 

In the following section, we will:
1) Study vertical plasma stability in elongated tokamak plasmas
2) Analyze Vertical Displacement Events (VDEs) - dangerous instabilities where the plasma moves vertically and can damage the vessel
3) Perform both linear and nonlinear analysis of plasma instabilities

To reduce output, we update settings to disable "performance monitoring" (eg. solver progress).

In [33]:
tokamaker.settings.pm=False
tokamaker.update_settings()

tokamaker.print_info()

Equilibrium Statistics:
  Topology                =   Limited
  Toroidal Current [A]    =    8.0000E+06
  Current Centroid [m]    =    4.876 -0.000
  Magnetic Axis [m]       =    4.900 -0.000
  Elongation              =    1.715 (U:  1.716, L:  1.715)
  Triangularity           =   -0.432 (U: -0.433, L: -0.431)
  Plasma Volume [m^3]     =   168.668
  q_0, q_95               =    1.280  2.948
  Peak Pressure [Pa]      =    1.4657E+06
  Stored Energy [J]       =    9.8792E+07
  <Beta_pol> [%]          =   75.5056
  <Beta_tor> [%]          =    0.8155
  <Beta_n>   [%]          =    1.1487
  Diamagnetic flux [Wb]   =    8.3240E-02
  Toroidal flux [Wb]      =    6.0425E+01
  l_i                     =    1.0112


### Compute linear stability as a function of $\beta_p$
Now we perform a scan of $\beta_p$, computing the linear stability properties of the equilibrium at each point. We do this by looping over desired values of $\beta_p$ and performing the following steps at each point:

 1. Compute the desired equilibrium by
   1. Set the shape constraints (needed as we remove them in step 3)
   2. Re-initialize $\psi$ (needed as we evolve far from the initial equilibrium in step 3)
   3. Solve for the equilibrium, with a few iterations to converge the desired $\beta_p$
 2. Compute linear stability using \ref OpenFUSIONToolkit.TokaMaker.eig_td() "eig_td()"
   1. Save most unstable mode and growth rate
 3. Compute nonlinear evolution of a perturbed equilibrium
   1. Set the initial condition by adding a small contribution from the most unstable linear mode
   2. Remove saddle and isoflux constraints
   3. Setup the time-dependent solver
   4. Loop over 30 timesteps with a timestep determined by the linear growth rate, saving the equilibrium and vertical position at each time

In [34]:
growth = []
beta_p = []
modes = []
zhist = []
beta_scale = 1.0

for beta_target in np.linspace(0.01,0.5,10): # how do you find the range???
    print('Computing Beta_approx [%] {0:.2f}'.format(beta_target*100.0))
    # Compute new equilibrium
    tokamaker.init_psi(major_radius,0.0,minor_radius,elongation,triangularity)
    beta_approx = beta_target*beta_scale

    for i in range(4):
        tokamaker.set_targets(Ip=Ip_target,Ip_ratio=(1.0/beta_approx - 1.0))
        tokamaker.solve()
        beta_approx *= beta_target/tokamaker.get_stats()['beta_pol']*100.0
    beta_scale = beta_approx/beta_target
    beta_p.append(tokamaker.get_stats()['beta_pol'])
    print('  Actual Beta_p = {0:.2f}'.format(beta_p[-1]))
    psi0 = tokamaker.get_psi(False)
    # Compute linear stability
    eig_vals, eig_vecs = tokamaker.eig_td(-1.E5,10,False)
    eig_sign = eig_vecs[0,tokamaker.r[:,1]>0.0][abs(eig_vecs[0,tokamaker.r[:,1]>0.0]).argmax()]
    modes.append(eig_vecs[0,:]*eig_sign)
    growth.append(eig_vals[0,0])
    # Compute nonlinear evolution
    psi_ic = psi0-0.01*eig_vecs[0,:]*(tokamaker.psi_bounds[1]-tokamaker.psi_bounds[0])/eig_sign
    tokamaker.set_psi(psi_ic)
    dt = 0.2/abs(eig_vals[0,0])
    tokamaker.setup_td(dt,1.E-13,1.E-11)
    sim_time = 0.0
    results = []
    z0 = [[sim_time,tokamaker.o_point[1]],]
    for i in range(30):
        sim_time, _, nl_its, lin_its, nretry = tokamaker.step_td(sim_time,dt)
        assert nretry >= 0
        z0.append([sim_time,tokamaker.o_point[1]])
        results.append(tokamaker.get_psi())
    zhist.append(z0)


Computing Beta_approx [%] 1.00


  Actual Beta_p = 1.00


Computing Beta_approx [%] 6.44


  Actual Beta_p = 6.46


Computing Beta_approx [%] 11.89


...

  Actual Beta_p = 39.13


Computing Beta_approx [%] 44.56


  Actual Beta_p = 44.62


Computing Beta_approx [%] 50.00


  Actual Beta_p = 50.04


### Plot growth rate trend
Once complete, we can plot the trend in the growth rate, which shows a decreasing growth rate for the vertical instability with increasing $\beta_p$.

In [35]:
fig, ax = plt.subplots(1,1)
ax.plot(beta_p,growth)
ax.set_ylim(top=0.0)
ax.grid(True)
ax.set_ylabel(r'$\gamma$ [1/s]')
_ = ax.set_xlabel(r'$\beta_p$ [%]')

save_figure(fig, "05_growth_rate_vs_beta_p.png")


Saved: examples/testing_1/05_growth_rate_vs_beta_p.png


### Plot linear eigenmodes

To better understand why this trend occurs, we can plot the linearly unstable mode structure as a function of $\beta_p$. From this we can see that as $\beta_p$ increases the mode structure shifts outboard and more of the perturbed flux (relative to the peak value) interacts with the wall. As the resistivity of the wall is what sets the timescale in a time-dependent Grad-Shafranov model, this explains the decrease in growth rate as as higher $\beta_p$ more flux must move through the wall (via resisitive diffusion) for the same growth in amplitude.

In [36]:
import matplotlib as mpl

norm = mpl.colors.Normalize(vmin=beta_p[0], vmax=beta_p[-1])
scalarMap = mpl.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis)

fig, ax = plt.subplots(1,4,sharey=True,constrained_layout=True,figsize=(8,4))
for ax_tmp in ax:
    tokamaker.plot_machine(fig,ax_tmp,limiter_color=None)
for j, i in enumerate((0,3,6,9)): # 0,3,6,9
    colorVal = scalarMap.to_rgba(beta_p[i])
    tokamaker.plot_psi(fig,ax[j],psi=modes[i],plasma_nlevels=6,normalized=False,plasma_color=[colorVal],opoint_color=None,xpoint_color=None,vacuum_nlevels=0)
    tokamaker.plot_eddy(fig,ax[j],dpsi_dt=modes[i]*abs(growth[i]),colormap='seismic',symmap=True,clabel=None)
for ax_tmp in ax:
    ax_tmp.set_xlabel(r'R [m]')
ax[0].set_ylabel(r'Z [m]')
_ = fig.colorbar(scalarMap,ax=ax[:],label=r'$\beta_p$ [%]')

save_figure(fig, "06_mode_structures_different_beta_p.png")

Saved: examples/testing_1/06_mode_structures_different_beta_p.png


### Plot nonlinear plasma evolution
We can also plot the evolution of the vertical position of the magnetic axis from the nonlinear evolution of each of the points in the $\beta_p$ scan. This shows the same behavior as the linear study (as expected of course), where the growth rate (velocity of the vertical position) decreases with increasing $\beta_p$. Additionally, clear linear (straight line on a log plot) and nonlinear phases are visible for each case.

In [37]:
fig, ax = plt.subplots(1,1)
for i, z0 in enumerate(zhist):
    z_hist = np.asarray(z0); z_hist = z_hist[1:,:] - [z_hist[1,0], z_hist[0,1]]
    colorVal = scalarMap.to_rgba(beta_p[i])
    ax.semilogy(z_hist[:,0]*1.E3,abs(z_hist[:,1]),color=colorVal)
ax.grid(True)
ax.set_ylabel(r'$|\Delta Z_0|$ [m]')
ax.set_xlabel(r'Time [ms]')
_ = fig.colorbar(scalarMap,ax=ax,label=r'$\beta_p$ [%]')

save_figure(fig, "07_nonlinear_plasma_evolution.png")


Saved: examples/testing_1/07_nonlinear_plasma_evolution.png


### Plasma Evolution During Vertical Displacement Event

We can visualize the time evolution of plasma flux surfaces during a vertical displacement event. Let's plots multiple plasma equilibria at different time steps, color-coded from the simulation time sequence, showing how the plasma shape and position change as the VDE progresses. The colorbar indicates the time progression in milliseconds, allowing you to see the plasma's vertical movement and deformation over the course of the instability.

In [38]:
fig, ax = plt.subplots(constrained_layout=True,figsize=(8,5))
tokamaker.plot_machine(fig,ax)
colors = plt.cm.jet(np.linspace(0,1,len(results)))
for i, result in enumerate(results):
    tokamaker.plot_psi(fig,ax,psi=result,plasma_nlevels=1,plasma_color=[colors[i]], vacuum_nlevels = 0,xpoint_color=None,opoint_color=None)
norm = mpl.colors.Normalize(vmin=0.0, vmax=sim_time*1.E3)
colors = plt.cm.jet(np.linspace(0,1,len(results)))

norm = mpl.colors.Normalize(vmin=0.0, vmax=sim_time*1.E3)
_ = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=plt.cm.jet),ax=ax,label='Time [ms]')

save_figure(fig, "08_vde_evolution.png")


Saved: examples/testing_1/08_vde_evolution.png


In [39]:
import matplotlib.animation
from matplotlib.ticker import FuncFormatter
from IPython.display import HTML
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['animation.embed_limit'] = 1.E8

plt.rcParams['lines.linewidth']=3
fig, ax = plt.subplots(figsize=(16,10))
times = np.linspace(0,sim_time, len(results))*1000
def animate(i):
    ax.clear()
    tokamaker.plot_machine(fig,ax)
    tokamaker.plot_psi(fig,ax,psi=results[i],plasma_nlevels=8,plasma_colormap= 'magma', vacuum_nlevels = 6,xpoint_color=None,opoint_color=None)
    ax.text(0.45, 0.72, f"{times[i]:.2f}"+' ms', color = 'k', fontsize = 18)
    ax.set_xlabel('R (m)', fontsize = 14)
    ax.set_ylabel('Z (m)', fontsize = 14)
    ax.set_ylim(-3.2, 3.2)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(results))

HTML(ani.to_jshtml())

# Save as GIF
writer = matplotlib.animation.PillowWriter(fps=5,
                                 metadata=dict(artist='Sophia Guizzo'),
                                 bitrate=1800)

# Save to simulation folder with user-defined name
gif_filename = "09_vde_evolution.gif"
gif_path = os.path.join(simulation_folder, gif_filename)
ani.save(gif_path, writer=writer)
print(f"Saved VDE animation: {gif_path}")

Saved VDE animation: examples/testing_1/09_vde_evolution.gif


In [40]:
# Final summary
print("\n" + "="*50)
print("SIMULATION COMPLETE")
print("="*50)
print(f"All files saved to: {simulation_folder}")
print("\nGenerated files:")
for file in os.listdir(simulation_folder):
    print(f"  - {file}")
print("="*50)


SIMULATION COMPLETE
All files saved to: examples/testing_1

Generated files:
  - 07_nonlinear_plasma_evolution.png
  - design_20250702_113339.json
  - 06_mode_structures_different_beta_p.png
  - 04_coil_currents.png
  - 08_vde_evolution.png
  - 05_growth_rate_vs_beta_p.png
  - 02_plasma_profiles.png
  - 01_vacuum_vessel_design.png
  - 03_initial_equilibrium.png
  - 09_vde_evolution.gif
